In [ ]:
!pip install gensim &> /dev/null
!pip install -U pip setuptools wheel &> /dev/null
!pip install texthero &> /dev/null
!pip install -U spacy &> /dev/null
!pip install natasha &> /dev/null
!pip install stop_words &> /dev/null
!pip install simplemma &> /dev/null

In [ ]:
!git clone https://github.com/stopwords-iso/stopwords-lv.git

Cloning into 'stopwords-lv'...
remote: Enumerating objects: 28, done.
remote: Total 28 (delta 0), reused 0 (delta 0), pack-reused 28
Unpacking objects: 100% (28/28), 5.77 KiB | 656.00 KiB/s, done.


In [ ]:
import string
import io
import os
import numpy as np
import pandas as pd
import re
import texthero
from re import sub
import nltk
from tqdm import tqdm
import simplemma
#import pymorphy2
#from pymorphy2.analyzer import MorphAnalyzer
import gensim
import pickle
from gensim.models import Word2Vec
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from stop_words import get_stop_words
import csv
import scipy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from tqdm import tqdm
from gensim.models import Word2Vec

#morph = pymorphy2.MorphAnalyzer()

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Mounted at /content/drive


In [ ]:
corpus_lv = []
with open(r'rainis_v20180716.txt') as file:
    for line in file:
        corpus_lv.append(line.rstrip())

In [ ]:
corpus_lv[:20]

["<doc source='http://www.korpuss.lv/klasika/Senie/Rainis/Talnzv/tnzvsat.htm' genre='Lirika' category='Lirika+Lirika|Tālas noskaņas zilā vakarā' title='Tālas noskaņas zilā vakarā'>",
 "<section source='http://www.korpuss.lv/klasika/Senie/Rainis/Talnzv/tnzv1.htm' title='Ieskaņa'>",
 'RAINIS',
 'TĀLAS NOSKAŅAS ZILĀ VAKARĀ',
 'Vēlreiz garā tuvajiem mīļā dzimtenē sirsnīgus sveicienus!',
 '...Daudz simtu jūdžu tāļumā,',
 'Aiz tīreļiem, purviem un siliem,',
 'Guļ mana dzimtene diendusā',
 'Tā aizsegta debešiem ziliem,',
 'Zil-saulainiem debešu palagiem',
 'Pret dvesmām un strāvām, un negaisiem...',
 'Bij dziļa ziema,',
 'Kad projām gāju',
 'Uz tāļu zemi,',
 'Uz svešu māju.',
 '«Nu miers virs zemes!»',
 'Man likās skanam,',
 '«Nu labs prāts cilvēkiem!»',
 'Pulksteņus zvanām;',
 'Un vējus es dzirdēju']

In [ ]:
stop_words_dict = []
with open(r'stopwords-lv.txt') as file:
    for line in file:
        stop_words_dict.append(line.rstrip())
stop_words_dict[:20]

['aiz',
 'ap',
 'apakš',
 'apakšpus',
 'ar',
 'arī',
 'augšpus',
 'bet',
 'bez',
 'bija',
 'biji',
 'biju',
 'bijām',
 'bijāt',
 'būs',
 'būsi',
 'būsiet',
 'būsim',
 'būt',
 'būšu']

In [ ]:
def preprocess_text(text_list, stop_words):
    # Remove elements with <> in them
    text_list = [x for x in text_list if "<" not in x and ">" not in x]
    
    # Make all elements lowercase
    text_list = [x.lower() for x in text_list]
    
    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    text_list = [x.translate(table) for x in text_list]
    
    # Get rid of double spaces
    text_list = [re.sub(' +', ' ', x) for x in text_list]
    
    # Get rid of stopwords
    text_list = [x for x in text_list if x not in stop_words_dict]

    text_list = [re.sub(r'\d+', '', x) for x in text_list]
    
    return text_list

preprocessed_text = preprocess_text(corpus_lv, stop_words_dict)
preprocessed_text[:20]

['rainis',
 'tālas noskaņas zilā vakarā',
 'vēlreiz garā tuvajiem mīļā dzimtenē sirsnīgus sveicienus',
 'daudz simtu jūdžu tāļumā',
 'aiz tīreļiem purviem un siliem',
 'guļ mana dzimtene diendusā',
 'tā aizsegta debešiem ziliem',
 'zilsaulainiem debešu palagiem',
 'pret dvesmām un strāvām un negaisiem',
 'bij dziļa ziema',
 'kad projām gāju',
 'uz tāļu zemi',
 'uz svešu māju',
 '«nu miers virs zemes»',
 'man likās skanam',
 '«nu labs prāts cilvēkiem»',
 'pulksteņus zvanām',
 'un vējus es dzirdēju',
 'ņirdzīgi smejam',
 'man paraustām svārkus']

In [ ]:
lemmatized_text = []
romanian_numbers = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]
for line in preprocessed_text:
    words = line.split()
    stemmed_line = []
    for word in words:
        normalized_word = simplemma.lemmatize(word, lang='lv')
        stemmed_line.append(normalized_word)
    lemmatized_text.append(" ".join(stemmed_line))
    
lemmatized_text = [s.replace("»", "").replace("«", "") for s in lemmatized_text if s.strip()]
lemmatized_text = [re.sub(r'\b\d+\b', '', string) for string in lemmatized_text if string not in romanian_numbers]


# print the lemmatized text
print(lemmatized_text[:20])


['rainis', 'tālais noskaņas zils vakars', 'vēlreiz gars tuvais mīļš dzimtene sirsnīgus sveiciens', 'daudz simtu jūdze tāļumā', 'aiz tīrelis purvs un sils', 'gulēt mans dzimtene diendusā', 'tā aizsegta debešiem zilais', 'zilsaulainde debešu palags', 'pret dvesmām un strāvām un negaiss', 'bij dziļš ziema', 'kad projām iet', 'uz tāļu zeme', 'uz svešs māja', 'nu miers virs zemes', 'es likties skanēt', 'nu labs prāts cilvēks', 'pulkstenis zvanām', 'un vējš es dzirdēt', 'ņirdzīgi smejam', 'es paraustām svārki']


In [ ]:
def save_list_to_csv(list_, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        for row in list_:
            writer.writerow([row])
save_list_to_csv(lemmatized_text, 'stemmed_text_lv_new.csv')

In [ ]:
lemmatized_text[:30]

['rainis',
 'tālais noskaņas zils vakars',
 'vēlreiz gars tuvais mīļš dzimtene sirsnīgus sveiciens',
 'daudz simtu jūdze tāļumā',
 'aiz tīrelis purvs un sils',
 'gulēt mans dzimtene diendusā',
 'tā aizsegta debešiem zilais',
 'zilsaulainde debešu palags',
 'pret dvesmām un strāvām un negaiss',
 'bij dziļš ziema',
 'kad projām iet',
 'uz tāļu zeme',
 'uz svešs māja',
 'nu miers virs zemes',
 'es likties skanēt',
 'nu labs prāts cilvēks',
 'pulkstenis zvanām',
 'un vējš es dzirdēt',
 'ņirdzīgi smejam',
 'es paraustām svārki',
 'un garš skrejam',
 'sniegt putens griezās',
 'es acs sist',
 'lēns uz Ziemsvētki',
 'eglītēm krist',
 'ak miers virs zeme',
 'un gars dusa',
 'un ziema ziema',
 'sirds sāpēt kust',
 'es sirds ziedons']

In [ ]:
stem_corp= pd.read_csv('stemmed_text_lv_new.csv')

In [ ]:
vectorizer = CountVectorizer(min_df=5)

data_vectorized = vectorizer.fit_transform(stem_corp['rainis'][::3])

In [ ]:
A = data_vectorized.toarray()
A = A.astype(float)

In [ ]:
feature_names = sorted(vectorizer.vocabulary_.keys())

In [ ]:
len(feature_names)

7948

In [ ]:
A.shape

(51055, 7948)

In [ ]:
K = 10
u, s, vt = scipy.sparse.linalg.svds(A.T, k=K)

In [ ]:
A.T.shape

(7948, 51055)

In [ ]:
u.shape, s.shape, vt.shape

((7948, 10), (10,), (10, 51055))

In [ ]:
with open('lv_vt_' + str(K) +  '.pickle', 'wb') as f:
  pickle.dump(vt, f)
with open('lv_u_' + str(K) + '.pickle', 'wb') as f:
  pickle.dump(u, f)
with open('lv_s_.' + str(K) + 'pickle', 'wb') as f:
  pickle.dump(s, f)

In [ ]:
print(feature_names[:50])

['aba', 'abi', 'abonents', 'abonēt', 'absolūti', 'aco', 'acs', 'actiņa', 'actiņas', 'actiņām', 'acumirklis', 'adata', 'adrese', 'advokatūrā', 'advokāts', 'adīt', 'afelds', 'aforisms', 'agrs', 'agrākais', 'agrāks', 'aicināt', 'aicināts', 'aidā', 'aila', 'ailā', 'aina', 'ainas', 'ainava', 'ainu', 'ainām', 'ainās', 'aita', 'aiva', 'aiz', 'aizbrauc', 'aizbrauca', 'aizbraucis', 'aizbraukt', 'aizbraukšu', 'aizbēg', 'aizdevums', 'aizdomas', 'aizdot', 'aizdzen', 'aizejošs', 'aizgrieža', 'aizgājis', 'aiziedams', 'aiziet']


In [ ]:
def preprocess_text(text):  
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = stop_words_dict
    tokens = [token for token in tokens if token not in stop_words]
    # Perform morphological analysis and lemmatization
    lemmatized_tokens = []
    for token in tokens:
        lemma = simplemma.lemmatize(token, lang='lv')
        lemmatized_tokens.append(lemma)
    # Join the preprocessed tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

def convertTextToEmbeddingSeries(text, dict_words, emb):
    wtext = preprocess_text(text)
    wdata = wtext.split(' ')
    word2ind = dict(zip(dict_words, np.arange(len(dict_words))))
    embText = []
    for word in wdata:
        ind = word2ind.get(word)
        if ind: embText.append(emb[ind])
    return np.array(embText)

In [ ]:
preprocessed_text = preprocess_text('Atnācu no ielas, atradu kaķi un sāku glāstīt viņa kažoku!')
preprocessed_text

'atnācu iela atrast kaķe sākt glāstīt viņa kažoks'

In [ ]:
textSeries = convertTextToEmbeddingSeries(preprocessed_text,
                                          feature_names,
                                          u)

In [ ]:
textSeries.shape

(7, 10)

In [ ]:
textSeries

array([[-7.44353949e-04,  5.46467283e-04, -9.69686382e-04,
         1.04959102e-03,  2.61089572e-04, -2.35160872e-04,
        -1.96949946e-05,  4.82218788e-04,  2.13832037e-04,
         7.74820069e-05],
       [-5.67993671e-04,  1.55899286e-03,  3.69818498e-04,
         2.85639079e-03, -9.51586479e-05, -4.91955126e-04,
         2.75490797e-04, -1.48344829e-04,  1.92822885e-03,
         9.08187656e-04],
       [-1.38086777e-05,  2.33192415e-03,  1.34745172e-03,
         1.30362757e-03,  3.17836809e-04, -4.88344403e-05,
         9.53449398e-05,  3.59607650e-03,  1.73060886e-03,
         1.83727405e-03],
       [-1.39013400e-03,  8.15709148e-05, -5.51433746e-04,
        -8.70441082e-05, -1.32905799e-03, -1.94448511e-04,
         1.64459824e-04,  2.70990863e-04,  3.21271392e-04,
         2.53756673e-04],
       [-1.92160472e-02, -5.81001224e-03, -8.32449303e-03,
         1.52680014e-02,  9.66549489e-03,  9.33756998e-03,
         5.24048927e-03,  1.47768684e-02,  9.10273327e-03,
         9.